In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')

In [3]:
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [4]:
os.chdir('C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\')
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [5]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P2_testing"
path = folder + participant
path

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2_testing'

In [6]:
EEG_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P2\EEGbaseline.csv', sep=";", index_col=0)

In [7]:
df_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P2\baseline.csv', sep=";", index_col=0)

In [9]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P2\\model.pkl", "rb"))

In [10]:
EEG_baseline

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_30_gamma,EEG_30_theta,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta
0,-1.440504e-10,1.166332e-10,2.606888e-10,5.463849e-09,-5.215928e-10,-4.847339e-10,-1.851318e-09,1.367583e-09,2.962264e-10,6.640914e-10,...,1.980897e-08,2.205318e-07,-2.325147e-11,9.996566e-09,-1.132049e-08,1.709258e-08,1.898458e-09,1.219276e-08,1.211897e-08,-1.933656e-08


In [11]:
df_baseline

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,80.231192,67.411985,95.232804,6.032405,0.752143,0.630035,0.890049,57.4689,30.3791,33,...,0.681868,415.0,140.847,0.33939,27.0,23.0,74.193548,87.096774,0.116537,254.024061


In [12]:
os.chdir(path)
(
    users,
    EEG_epochs,
    EEG_filtered,
    Signals_epochs,
    Signals,
    EEG_dict,
    features_signals,
    features_EEG,
    features_epochs,
    features_epochs_EEG,
) = ({}, {}, {}, {}, {}, {}, {}, {}, {}, {})

for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [13]:
users

{'P2_testing.xdf': [{'info': defaultdict(list,
               {'name': ['PsychoPy Markers'],
                'type': ['Markers'],
                'channel_count': ['2'],
                'channel_format': ['string'],
                'source_id': ['Example'],
                'nominal_srate': ['0.000000000000000'],
                'version': ['1.100000000000000'],
                'created_at': ['2178.796828400000'],
                'uid': ['b24f4eb1-e7a8-4a27-9dce-0b79713b5341'],
                'session_id': ['default'],
                'hostname': ['DESKTOP-8HVM7BE'],
                'v4address': [None],
                'v4data_port': ['16573'],
                'v4service_port': ['16573'],
                'v6address': [None],
                'v6data_port': ['16573'],
                'v6service_port': ['16573'],
                'desc': [None],
                'stream_id': 5,
                'effective_srate': 0}),
   'footer': {'info': defaultdict(list,
                {'first_timestamp'

In [14]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16

In [15]:
sensors = ["ECG", "EDA", "RESP"]
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

    data[user.split(".")[0]]["Markers"].pop(0)
    data[user.split(".")[0]]["Markers"].pop(-1)
    data[user.split(".")[0]]["Markers Timestamps"].pop(0)
    data[user.split(".")[0]]["Markers Timestamps"].pop(-1)

onset, offset, videos, valence, arousal = ({}, {}, {}, {}, {})
for user in data.keys():
    valence[user] = data[user]["Valence"]
    arousal[user] = data[user]["Arousal"]
    onset[user], offset[user], videos[user] = getMarkers(
        data[user]["Markers"], data[user]["Markers Timestamps"]
    )

onset_index = {}
offset_index = {}
onset_index_EEG = {}
offset_index_EEG = {}

for user in data.keys():
    onset_index[user], offset_index[user] = getMarkersIndex(
        onset[user], offset[user], data[user]["Signals"]["Time"]
    )
    onset_index_EEG[user], offset_index_EEG[user] = getMarkersIndex(
        onset[user], offset[user], np.array(data[user]["EEG"]["Time"])
    )

events_diff = {}

for keys in onset.keys():
    events_diff[keys] = CalculateEventsDiff(onset[keys], offset[keys])

In [16]:
"""EEG Processing"""

EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)
EEG_epochs = getEpochs(EEG_filtered, onset_index_EEG, events_diff, EEG_fs)
EEG_dict = getVideosDict(EEG_epochs, videos)
features_EEG = getEEGBands(EEG_dict, EEG_fs)

In [17]:
features_EEG

{'P2_testing': {'EMDB/Horror/1009.avi': {'EEG_1': {'alpha': 7.252007221625367e-10,
    'betha': 6.807970019244695e-10,
    'gamma': 5.312112856190948e-10,
    'theta': 2.937498949678456e-09},
   'EEG_2': {'alpha': 7.912955571978788e-10,
    'betha': 8.278322904054835e-10,
    'gamma': 5.861773877319425e-10,
    'theta': 1.6866157919486553e-09},
   'EEG_3': {'alpha': 5.569739445516107e-10,
    'betha': 8.506359435177046e-10,
    'gamma': 6.947382490796424e-10,
    'theta': 1.1646304010336297e-09},
   'EEG_4': {'alpha': 7.694279439872504e-10,
    'betha': 8.974828596370613e-10,
    'gamma': 6.45226530470481e-10,
    'theta': 1.2730532141292636e-09},
   'EEG_5': {'alpha': 6.868929866174321e-10,
    'betha': 7.184823857319328e-10,
    'gamma': 4.70068922577943e-10,
    'theta': 1.1339357285506875e-09},
   'EEG_6': {'alpha': 6.17770567812413e-10,
    'betha': 2.3564367867020224e-09,
    'gamma': 2.5939799160829224e-09,
    'theta': 9.446740780652828e-10},
   'EEG_7': {'alpha': 5.72760852850

In [18]:
EEG_list = EEG_baseline.loc[0].values.flatten().tolist()

In [19]:
Signals_epochs = getSignalsEpochs(data, onset_index, events_diff, Opensignals_fs)
Signals = getVideosDict(Signals_epochs, videos)
features_signals = getFeatures(Signals, Opensignals_fs, resolution)

C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\neurokit2\rsp\rsp_rrv.py:212: RuntimeWarning: invalid value encountered in sqrt
  out["SD2"] = np.sqrt(2 * np.std(bb

In [20]:
columns = list()
category, video = [], []

for key in features_epochs_EEG.keys():
    user = key
    print(user)
videos = list(features_EEG[user].keys())
for epochs in features_EEG[user][videos[0]].keys():
    for bands in features_EEG[user][videos[0]][epochs].keys():
        columns.append(epochs + "_" + bands)
df_EEG = pd.DataFrame(columns=columns)

for users in features_EEG.keys():
    temp_df = pd.DataFrame(columns=columns)

    for i, videos in enumerate(features_EEG[users].keys()):
        if videos != "baseline":
            temp_list = list()
            for channel in features_EEG[users][videos].keys():
                for band in features_EEG[users][videos][channel].keys():
                    temp_list.append(
                        features_EEG[users][videos][channel][band]
                    )
            temp_df.loc[i] = list(np.array(temp_list)-np.array(EEG_list))
            category.append(videos.split("/")[1])
            video.append(videos.split("/")[2])

    df_EEG = pd.concat([df_EEG, temp_df], ignore_index=True)
df_EEG["Category"] = category
df_EEG["Video"] = video

In [21]:
df_EEG

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta,Category,Video
0,8.692511e-10,5.641638e-10,2.705224e-10,-2.526350e-09,1.312888e-09,1.312566e-09,2.437495e-09,3.190323e-10,2.607476e-10,1.865445e-10,...,3.697225e-08,1.780203e-08,4.477445e-08,6.602265e-08,3.081054e-08,1.148363e-08,1.367492e-09,1.494888e-07,Horror,1009.avi
1,1.779516e-09,8.462600e-10,3.993072e-10,1.829521e-07,3.548017e-09,1.501141e-09,2.545096e-09,4.499460e-07,1.819125e-09,4.794578e-10,...,4.179946e-08,2.802772e-08,5.708964e-08,2.996943e-07,3.681703e-08,2.195602e-08,1.424649e-08,3.310180e-07,Horror,1007.avi
2,1.097871e-03,8.682479e-04,2.474952e-04,2.884208e-03,1.097138e-03,8.680843e-04,2.476973e-04,2.881661e-03,1.096845e-03,8.684547e-04,...,1.114930e-03,8.671750e-04,2.408129e-04,2.956409e-03,1.086872e-03,8.450382e-04,2.343343e-04,2.886324e-03,Horror,1008.avi
3,8.607619e-10,4.251433e-10,8.657054e-11,9.107159e-09,1.351993e-09,1.140133e-09,2.222084e-09,1.204792e-08,2.926836e-10,1.156468e-11,...,4.143797e-08,2.080052e-08,4.594731e-08,6.646742e-08,3.601858e-08,1.521710e-08,2.499706e-09,1.541301e-07,Horror,1006.avi
4,2.341193e-03,7.542431e-04,2.761040e-04,2.489216e-03,2.340012e-03,7.542928e-04,2.762973e-04,2.487447e-03,2.339569e-03,7.544893e-04,...,2.310055e-03,7.397155e-04,2.602509e-04,2.469076e-03,2.324935e-03,7.498291e-04,2.669040e-04,2.460861e-03,Horror,1003.avi
5,1.213481e-09,5.039310e-10,1.724322e-10,-3.953017e-11,1.768118e-09,1.336725e-09,2.388392e-09,1.667459e-09,5.932023e-10,3.413376e-10,...,5.004530e-08,2.725465e-08,4.871132e-08,8.372814e-08,4.688265e-08,1.955093e-08,7.796275e-09,1.488172e-07,Social Negative,3009.avi
6,9.523124e-10,3.959577e-10,4.899445e-11,-6.102155e-10,1.408910e-09,1.133132e-09,2.144419e-09,1.443581e-09,3.633683e-10,-7.023795e-11,...,4.593170e-08,1.712237e-08,3.642241e-08,8.550032e-08,4.037866e-08,1.142881e-08,-2.818920e-09,1.592825e-07,Social Negative,3003.avi
7,1.258374e-09,3.938257e-10,2.454835e-11,-4.136322e-10,2.004071e-09,1.113790e-09,2.106790e-09,1.915922e-09,6.592944e-10,-7.119373e-11,...,6.059451e-08,1.809708e-08,3.683710e-08,9.262850e-08,4.413642e-08,9.138391e-09,-4.558515e-09,1.534049e-07,Social Negative,3007.avi
8,1.270149e-09,5.717860e-10,1.387886e-10,-1.265532e-09,1.688585e-09,1.239215e-09,2.290986e-09,8.770743e-10,5.844988e-10,1.877103e-10,...,5.557621e-08,2.247705e-08,4.275848e-08,7.270877e-08,5.629808e-08,1.765162e-08,1.763482e-09,1.399898e-07,Social Negative,3006.avi
9,9.139431e-10,6.229849e-10,2.909233e-10,-3.554736e-10,1.492936e-09,1.390443e-09,2.363391e-09,1.309083e-09,3.446821e-10,1.640409e-10,...,4.860765e-08,2.150839e-08,4.438890e-08,8.042541e-08,4.906034e-08,1.529369e-08,1.112942e-09,1.595590e-07,Social Negative,3008.avi


In [22]:
category, videos = [], []

for key in features_signals.keys():
    user = key
video = list(features_signals[user].keys())
df = pd.DataFrame(columns=features_signals[user][video[0]].columns)
for users in features_signals.keys():
    for epoch in features_signals[users].keys():
        if epoch != "baseline":
            df = pd.concat([df, features_signals[users][epoch].sub(df_baseline)], ignore_index=True)
            category.append(epoch.split("/")[1])
            videos.append(epoch.split("/")[2])
df["Category"] = category
df["Video"] = videos

In [23]:
df

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,1.287265,4.859401,-0.018121,-0.604866,-0.012922,0.000120,-0.059845,-9.4082,-0.7666,-27,...,-80.0604,-0.224699,-22.0,-18.0,9.139785,-3.763441,NaN,-134.668500,Horror,1009.avi
1,-0.698508,3.998875,-0.018324,-0.127768,0.006281,0.000121,-0.049841,-3.1973,-0.4384,-26,...,-124.5384,-0.306935,-25.0,-21.0,-24.193548,-37.096774,NaN,-37.295736,Horror,1007.avi
2,-1.822357,-5.571744,-1.506189,2.436575,0.022451,0.010125,0.080193,29.8721,4.5932,-26,...,-44.4780,-0.137782,-22.0,-18.0,9.139785,-3.763441,NaN,-125.309648,Horror,1008.avi
3,-0.534785,2.338037,-4.346412,-0.411379,0.004517,0.030130,-0.029835,-3.4466,0.3857,-27,...,105.2646,0.072857,-24.0,-20.0,0.806452,-12.096774,NaN,-22.329938,Horror,1006.avi
4,-2.931361,2.338175,-9.539751,-2.535566,0.025650,0.070138,-0.029836,-22.1580,-11.9687,-33,...,-68.9409,-0.164010,-22.0,-19.0,-7.526882,-3.763441,NaN,-87.050560,Horror,1003.avi
5,2.801985,1.536457,4.742437,1.139238,-0.024370,-0.029886,-0.019834,2.7597,-4.2741,-29,...,-108.2298,-0.277262,-23.0,-20.0,-14.193548,-7.096774,NaN,-155.978153,Social Negative,3009.avi
6,-1.084957,5.741200,-10.746027,-2.984300,0.007078,0.080135,-0.069853,-28.0929,-11.5357,-32,...,9.6369,-0.114452,-24.0,-20.0,0.806452,-12.096774,NaN,-13.936230,Social Negative,3003.avi
7,2.950186,8.519273,-1.505158,-1.517567,-0.028756,0.010118,-0.099861,-19.2355,-10.0139,-33,...,-81.5430,-0.212129,-24.0,-22.0,-54.193548,-27.096774,NaN,-216.182285,Social Negative,3007.avi
8,2.440138,1.536325,6.436738,1.281204,-0.020849,-0.039888,-0.019832,5.4302,-1.2333,-26,...,-131.2101,-0.320858,-27.0,-23.0,-74.193548,-87.096774,NaN,-249.210885,Social Negative,3006.avi
9,4.737415,8.518685,1.517565,-0.526440,-0.043079,-0.009882,-0.099855,-12.2969,-6.1104,-30,...,-54.8562,-0.141255,-22.0,-19.0,-7.526882,-3.763441,NaN,-26.959643,Social Negative,3008.avi


In [24]:
"""Concatenate Dataframes"""
columns = df.columns[: (len(df.columns) - 2)]
columns_EEG = df_EEG.columns[: (len(df_EEG.columns) - 2)]
full_dataframe = pd.concat([df_EEG[columns_EEG], df], axis=1)
full_columns = full_dataframe.columns[: (len(full_dataframe.columns)-2)]

In [25]:
full_dataframe

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,8.692511e-10,5.641638e-10,2.705224e-10,-2.526350e-09,1.312888e-09,1.312566e-09,2.437495e-09,3.190323e-10,2.607476e-10,1.865445e-10,...,-80.0604,-0.224699,-22.0,-18.0,9.139785,-3.763441,NaN,-134.668500,Horror,1009.avi
1,1.779516e-09,8.462600e-10,3.993072e-10,1.829521e-07,3.548017e-09,1.501141e-09,2.545096e-09,4.499460e-07,1.819125e-09,4.794578e-10,...,-124.5384,-0.306935,-25.0,-21.0,-24.193548,-37.096774,NaN,-37.295736,Horror,1007.avi
2,1.097871e-03,8.682479e-04,2.474952e-04,2.884208e-03,1.097138e-03,8.680843e-04,2.476973e-04,2.881661e-03,1.096845e-03,8.684547e-04,...,-44.4780,-0.137782,-22.0,-18.0,9.139785,-3.763441,NaN,-125.309648,Horror,1008.avi
3,8.607619e-10,4.251433e-10,8.657054e-11,9.107159e-09,1.351993e-09,1.140133e-09,2.222084e-09,1.204792e-08,2.926836e-10,1.156468e-11,...,105.2646,0.072857,-24.0,-20.0,0.806452,-12.096774,NaN,-22.329938,Horror,1006.avi
4,2.341193e-03,7.542431e-04,2.761040e-04,2.489216e-03,2.340012e-03,7.542928e-04,2.762973e-04,2.487447e-03,2.339569e-03,7.544893e-04,...,-68.9409,-0.164010,-22.0,-19.0,-7.526882,-3.763441,NaN,-87.050560,Horror,1003.avi
5,1.213481e-09,5.039310e-10,1.724322e-10,-3.953017e-11,1.768118e-09,1.336725e-09,2.388392e-09,1.667459e-09,5.932023e-10,3.413376e-10,...,-108.2298,-0.277262,-23.0,-20.0,-14.193548,-7.096774,NaN,-155.978153,Social Negative,3009.avi
6,9.523124e-10,3.959577e-10,4.899445e-11,-6.102155e-10,1.408910e-09,1.133132e-09,2.144419e-09,1.443581e-09,3.633683e-10,-7.023795e-11,...,9.6369,-0.114452,-24.0,-20.0,0.806452,-12.096774,NaN,-13.936230,Social Negative,3003.avi
7,1.258374e-09,3.938257e-10,2.454835e-11,-4.136322e-10,2.004071e-09,1.113790e-09,2.106790e-09,1.915922e-09,6.592944e-10,-7.119373e-11,...,-81.5430,-0.212129,-24.0,-22.0,-54.193548,-27.096774,NaN,-216.182285,Social Negative,3007.avi
8,1.270149e-09,5.717860e-10,1.387886e-10,-1.265532e-09,1.688585e-09,1.239215e-09,2.290986e-09,8.770743e-10,5.844988e-10,1.877103e-10,...,-131.2101,-0.320858,-27.0,-23.0,-74.193548,-87.096774,NaN,-249.210885,Social Negative,3006.avi
9,9.139431e-10,6.229849e-10,2.909233e-10,-3.554736e-10,1.492936e-09,1.390443e-09,2.363391e-09,1.309083e-09,3.446821e-10,1.640409e-10,...,-54.8562,-0.141255,-22.0,-19.0,-7.526882,-3.763441,NaN,-26.959643,Social Negative,3008.avi


In [26]:
X = np.array(full_dataframe[full_columns])
Y = np.array(full_dataframe[["Category"]])

In [27]:
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
print(prediction)
print(full_dataframe[["Category"]])
from sklearn import metrics
print("Accuracy = " + str(metrics.accuracy_score(np.array(full_dataframe[["Category"]]),prediction)*100)+"%")

['Social Negative' 'Social Negative' 'Horror' 'Social Negative' 'Horror'
 'Social Negative' 'Social Negative' 'Social Negative' 'Social Negative'
 'Social Negative']
          Category
0           Horror
1           Horror
2           Horror
3           Horror
4           Horror
5  Social Negative
6  Social Negative
7  Social Negative
8  Social Negative
9  Social Negative
Accuracy = 70.0%


In [8]:
output = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P0\output_test.csv', sep=",", index_col=False)
features = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P0\final_features.csv', sep=",", index_col=False)

In [9]:
output

,Variable,True Value,Prediction
0,Category,Horror,Horror
1,Category,Horror,Horror
2,Category,Horror,Horror
3,Category,Horror,Horror
4,Category,Horror,Horror
...,...,...,...
1930,Category,Social Negative,Horror
1931,Category,Social Negative,Horror
1932,Category,Social Negative,Horror
1933,Category,Social Negative,Horror


In [10]:
features

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,0.000002,7.930818e-07,0.000810,0.000007,0.000023,0.000004,0.019973,0.000075,3.330531e-05,1.263925e-05,...,-0.214109,-18.0,-28.9107,-0.072311,-28.0,-19.0,-45.238095,-23.015873,NaN,-64.733583
1,0.000002,7.888573e-07,0.000813,0.000007,0.000023,0.000005,0.019944,0.000085,3.340273e-05,1.267303e-05,...,-0.215185,-18.0,-34.8411,-0.088923,-29.0,-19.0,-45.238095,-34.126984,NaN,-64.993177
2,0.000002,7.871184e-07,0.000820,0.000008,0.000023,0.000004,0.019944,0.000100,3.337410e-05,1.257313e-05,...,-0.212341,-18.5,-42.2541,-0.109600,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.951388
3,0.000002,7.817731e-07,0.000828,0.000007,0.000022,0.000004,0.019956,0.000074,3.297560e-05,1.231116e-05,...,-0.212341,-18.5,-42.2541,-0.109600,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.951388
4,0.000002,7.893067e-07,0.000835,0.000008,0.000022,0.000004,0.019957,0.000100,3.317275e-05,1.225755e-05,...,-0.211728,-17.5,-41.5128,-0.107701,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.771889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,0.000006,2.195289e-06,0.002339,0.000019,0.000018,0.000003,0.008400,0.000047,4.621875e-07,1.515247e-07,...,-0.196652,-6.0,-42.2541,-0.111713,-30.0,-17.0,-23.015873,-45.238095,NaN,-59.773022
1931,0.000006,2.209341e-06,0.002385,0.000016,0.000018,0.000003,0.008521,0.000050,4.660621e-07,1.532851e-07,...,-0.191835,-6.0,-42.2541,-0.111713,-30.0,-17.0,-23.015873,-45.238095,NaN,-58.470722
1932,0.000006,2.244610e-06,0.002437,0.000018,0.000018,0.000003,0.008647,0.000052,4.716830e-07,1.557388e-07,...,-0.206949,-5.0,-48.1845,-0.127904,-31.0,-18.0,-32.738095,-53.571429,NaN,-62.333989
1933,0.000006,2.273048e-06,0.002494,0.000018,0.000018,0.000003,0.008785,0.000045,4.789261e-07,1.577481e-07,...,-0.209765,-5.0,-48.1845,-0.127904,-31.0,-18.0,-32.738095,-53.571429,NaN,-63.112704


In [11]:
from sklearn import metrics
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")
# print("Simulated Accuracy = " + str(metrics.accuracy_score(np.array(simulate["True Value"]),np.array(simulate["Prediction"]))*100)+"%")

Real Accuracy = 46.40826873385013%


In [13]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\model.pkl", "rb"))

In [14]:
X = np.array(features)
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
print("Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),prediction)*100)+"%")

Accuracy = 46.40826873385013%
